# EDA: данные из московского кредитного банка (МКБ)

In [2]:
import pandas as pd
import numpy as np

# Загрузка данных

In [7]:
tmp = pd.read_csv('data/lightautoml.csv')
tmp.head()

,id_contract;TARGET
0,17892;0.020072278
1,17893;0.04022719
2,17894;0.049487215
3,17895;0.20958135
4,17896;0.97004336


In [4]:
data_train = pd.read_csv('data/train_dataset_hackathon_mkb.csv', encoding='cp1251', delimiter=';')
data_test = pd.read_csv('data/test_dataset_hackathon_mkb.csv', encoding='cp1251', delimiter=';')

data_train.shape, data_test.shape

((17891, 124), (7330, 123))

# Смотрим на данные

In [8]:
data_train.sample(5)

,id_contract,id_client,SIGN_DATE,IP_flag,TARGET,F1100,F1110,F1150,F1160,F1170,...,WINNERNUMBER_95_EVER,SIGNEDNUMBER_95_EVER,SUM_95_EVER,FLAG_DISQUALIFICATION,COUNT_CHANGE_YEAR,COUNT_CHANGE_EVER,BIRTHDATE,AGE,SEX_NAME,CITIZENSHIP_NAME
5574,5575,10500,24MAY2018:00:00:00,0,0,2321000.0,NaN,2321000.0,NaN,NaN,...,59.0,59.0,265900132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5162,5163,9724,14MAY2018:00:00:00,0,0,337618000.0,299000.0,336206000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6687,6688,10626,19JUN2018:00:00:00,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17561,17562,3496,29MAR2019:00:00:00,0,0,612492000.0,1055000.0,120552000.0,NaN,490885000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10180,10181,9358,24SEP2018:00:00:00,0,1,213000.0,NaN,213000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data_test.sample(5)

,id_contract,id_client,SIGN_DATE,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,...,WINNERNUMBER_95_EVER,SIGNEDNUMBER_95_EVER,SUM_95_EVER,FLAG_DISQUALIFICATION,COUNT_CHANGE_YEAR,COUNT_CHANGE_EVER,BIRTHDATE,AGE,SEX_NAME,CITIZENSHIP_NAME
6620,24512,5586,18NOV2019:00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,18830,3780,06MAY2019:00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13APR1982:00:00:00,37.0,женский,Российская Федерация
3439,21331,8184,16JUL2019:00:00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,...,380.0,374.0,49363710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1491,19383,10894,24MAY2019:00:00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
691,18583,3452,25APR2019:00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,01SEP1986:00:00:00,33.0,мужской,Российская Федерация


Заметим, что столбец `SIGN_DATE` содержит праздничные даты, хотя банк не работает по праздникам. Видимо, это даты отложенных транзакций

Посмотрим, как различаются тестовая и тренировочная выборки по клиентам

In [9]:
A1 = set(data_train['id_client'].values)
A2 = set(data_test['id_client'].values)
print('Число общих клиентов (и в test, и в train):', len(A1&A2))
print('Число клиентов, которые только в test:', len(A2-A1))

Число общих клиентов (и в test, и в train): 948
Число клиентов, которые только в test: 3813


Посмотрим на распределение по классам

In [10]:
data_train.TARGET.value_counts()

0    10502
1     7389
Name: TARGET, dtype: int64

Есть столбец `IP_flag`, который свидетельствует о плохом/хорошем контракте. Логично предположить, что этот столбец связан с целевым признаком

In [11]:
pd.crosstab(index=data_train.IP_flag,
            columns=data_train.TARGET)

TARGET,0,1
IP_flag,,
0,8214,6177
1,2288,1212


Как мы видим, когда `IP_flag` равен 1, то пропорции классов **почти 2 к 1**. При `IP_flag` = 0 пропорция уже другая - 8 к 6